In [ ]:
import MMGPD
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import quad
import vegas
import time
import pandas as pd

In [ ]:

def perform_vegas_integration(myAnalysis, t, xi, nitn=35, neval=1_000):
    """
    Perform VEGAS integration for a range of t and xi values.

    Parameters:
        myAnalysis: An object containing the method `xGPDxi` for GPD calculations.
        t_values: A list or array of t values.
        xi_values: A list or array of xi values.
        nitn: Number of VEGAS iterations (default: 35).
        neval: Number of evaluations per iteration (default: 1000).

    Returns:
        results: A dictionary with (t, xi) as keys and (mean, sdev) as values.
        elapsed_time: Total elapsed time for the computation.
    """

    # Define the integrand for VEGAS
    def integrand(x, t, xi):
        # Handle singularities by avoiding exact evaluation at x = xi
        if np.isclose(x[0], xi, atol=1e-9) or np.isclose(x[0], -xi, atol=1e-9):
            return 0
        else:
            term1 = (
                (myAnalysis.xGPDxi("Set11", "H", "uv", x[0], t, xi)/x[0] +
                 2 * myAnalysis.xGPDxi("Set11", "H", "ubar", x[0], t, xi)/x[0]) *
                np.power(2 / 3, 2) *
                (1 / (xi - x[0]) - 1 / (xi + x[0]))
            )
            term2 = (
                (myAnalysis.xGPDxi("Set11", "H", "dv", x[0], t, xi)/x[0] +
                 2 * myAnalysis.xGPDxi("Set11", "H", "dbar", x[0], t, xi)/x[0]) *
                np.power(-1 / 3, 2) *
                (1 / (xi - x[0]) - 1 / (xi + x[0]))
            )
            return term1 + term2

    # Wrapper for VEGAS
    def vegas_integrand(x, t, xi):
        return integrand(x, t, xi)

    # Initialize results
    ReH_Results = {}

    # Perform VEGAS integration for all t and xi values
    
        
    # Initialize VEGAS integrator
    integ = vegas.Integrator([[0, 1]])  # Integration bounds for x: 0 to 1
            
    # Run the integration
    result = integ(lambda x: vegas_integrand(x, t, xi), nitn=nitn, neval=neval)
            
    # Store the mean and standard deviation in the results
    ReH_Results[(t, xi)] = (result.mean, result.sdev)

    
    return ReH_Results


In [ ]:
# Define the integrand as a function of xi
def integrand(x, xi):
    if abs(xi) < abs(x[0]):  # Apply the Heaviside function condition
        return 0.0
    x_val = x[0]  # Vegas passes variables as an array
    term1 = 1 / (xi - x_val)
    term2 = 1 / (xi + x_val)
    prefactor = (1 - (x_val / xi)**2) * 3 * (x_val / xi)
    return (term1 - term2) * prefactor

# Define the vegas wrapper for a specific xi
def vegas_integrand(x, xi):
    return integrand(x, xi)

# Function to perform integration for a specific xi
def integrate_for_xi(xi):
    integ = vegas.Integrator([[0, 1]])  # Create the vegas integrator
    
    # Define a wrapped integrand that includes xi
    def wrapped_integrand(x):
        return vegas_integrand(x, xi)
    
    # Run the integration
    result = integ(wrapped_integrand, nitn=500, neval=10_000)
    return result.mean, result.sdev


In [ ]:
myAnalysis = MMGPD.GPDAnalysis("HGAG23")

In [ ]:
HallA = pd.read_csv('CFF-DataInput.dat',delim_whitespace=True)

In [ ]:
for items in HallA["xi"]:
    print("D-Term,xi: ",items,",",integrate_for_xi(items))



In [ ]:
HallAReH = []
for index in range(len(HallA["xi"])):
    HallAReH.append(perform_vegas_integration(myAnalysis, HallA["t"][index], HallA["xi"][index], nitn=35, neval=100))
HallAReH

In [ ]:
# Flatten the list of dictionaries into a list of tuples
dataHallAReH = [
    (t, xi, mean, sdev) 
    for item in HallAReH 
    for (t, xi), (mean, sdev) in item.items()
]

# Create a DataFrame
df = pd.DataFrame(dataHallAReH, columns=["t", "xi", "ReH", "Unc"])

# Save the DataFrame to a CSV file
df.to_csv('CFFOutput.csv', index=False)
